In [2]:
pip install lifetimes

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install sqlalchemy

Note: you may need to restart the kernel to use updated packages.


In [74]:
import lifetimes
from sqlalchemy import create_engine
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt
from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter
from lifetimes.plotting import plot_period_transactions
from lifetimes.plotting import plot_frequency_recency_matrix
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [122]:
df_ = pd.read_excel("https://github.com/Prenagen24/Online-Retail-Customer-Value-Prediction/raw/main/OnlineRetail.xlsx")
df = df_.copy()
df.shape
OUT = (541909, 8)
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,541909.0,9.552250,218.081158,-80995.00,1.00,3.00,10.00,80995.0
UnitPrice,541909.0,4.611114,96.759853,-11062.06,1.25,2.08,4.13,38970.0
CustomerID,406829.0,15287.690570,1713.600303,12346.00,13953.00,15152.00,16791.00,18287.0


In [196]:
#Drop NULL dan Dupllicate
df_clean = df
df_clean.dropna(inplace=True)
df_clean.drop_duplicates(inplace=True)
#Drop Quantity < 1
df_clean.drop(df_clean[df_clean['Quantity'] <= 0].index, inplace=True)
#Drop Unit Price < 1
df_clean.drop(df_clean[df_clean['UnitPrice'] <= 0].index, inplace=True)
#Drop InvoiceNO cancel
df_clean.drop(df_clean[df_clean['InvoiceNo'].astype('str').str.startswith('C', na=False)].index, inplace=True)
#Drop Stock Code that has length longer than 6 characters
invalidStockCode = ["BANK CHARGES", "C2", "CRUK", "D", "DOT", "M", "PADS", "POST", "S", "gift_0001_10", "gift_0001_20", "gift_0001_30", "gift_0001_40", "gift_0001_50", "gift_0001_70", "gift_0001_80", "gift_0001_90", "gif"]
df_clean.drop(df_clean[df_clean['StockCode'].isin(invalidStockCode)].index, inplace=True)

In [197]:
df_clean

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Total Price
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
...,...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France,10.20
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France,12.60
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France,16.60
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France,16.60


In [198]:
#Calculate Total Price
df_clean['Total Price'] = df_clean['UnitPrice'] * df_clean['Quantity']
df_clean.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Total Price
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34


In [199]:
#Implement RFM
clv = lifetimes.utils.summary_data_from_transaction_data(
    df_clean,'CustomerID','InvoiceDate','Total Price',observation_period_end='2011-12-09')

clv

,frequency,recency,T,monetary_value
CustomerID,,,,
12346.0,0.0,0.0,325.0,0.000000
12347.0,6.0,365.0,367.0,599.701667
12348.0,3.0,283.0,358.0,261.480000
12349.0,0.0,0.0,18.0,0.000000
12350.0,0.0,0.0,310.0,0.000000
...,...,...,...,...
18280.0,0.0,0.0,277.0,0.000000
18281.0,0.0,0.0,180.0,0.000000
18282.0,1.0,119.0,126.0,77.840000


In [205]:
clv = clv[clv['frequency']>0] # we want only customers shopped more than 2 times
clv.head()

,frequency,recency,T,monetary_value,expected_purc_3_months
CustomerID,,,,,
12347.0,6.0,365.0,367.0,599.701667,1.424030
12348.0,3.0,283.0,358.0,261.480000,0.793040
12352.0,6.0,260.0,296.0,181.540000,1.716594
12356.0,2.0,303.0,325.0,269.905000,0.622640
12358.0,1.0,149.0,150.0,523.200000,0.717645


In [206]:
#Build the BG/NBD Model
bgf = BetaGeoFitter(penalizer_coef=0.1)
bgf.fit(clv['frequency'], clv['recency'], clv['T'])

<lifetimes.BetaGeoFitter: fitted with 2776 subjects, a: 0.00, alpha: 55.07, b: 0.05, r: 0.98>

In [207]:
#Predict The 3 Months Purchase
t = 90 # 30 day period
clv['expected_purc_3_months'] = bgf.conditional_expected_number_of_purchases_up_to_time(t, clv['frequency'], clv['recency'], clv['T'])
clv.sort_values(by='expected_purc_3_months',ascending=False).head()

,frequency,recency,T,monetary_value,expected_purc_3_months
CustomerID,,,,,
14911.0,129.0,372.0,373.0,1051.600388,27.322294
12748.0,112.0,373.0,373.0,282.552054,23.748762
17841.0,111.0,372.0,373.0,360.032072,23.538446
15311.0,89.0,373.0,373.0,676.210899,18.913822
14606.0,87.0,372.0,373.0,134.802644,18.493315


In [210]:
#Check the Correlation between Frequency and Monetary Value (Preferred Low)
clv[['frequency','monetary_value']].corr()

,frequency,monetary_value
frequency,1.000000,0.015763
monetary_value,0.015763,1.000000


In [211]:
#Build the Gamma Gamma Model
ggf = GammaGammaFitter(penalizer_coef=0.1)
ggf.fit(clv["frequency"],
        clv["monetary_value"])

<lifetimes.GammaGammaFitter: fitted with 2776 subjects, p: 1.03, q: 0.18, v: 0.93>

In [212]:
#Predict the 3 Motnths CLV
clv['3_months_clv']=ggf.customer_lifetime_value(bgf,
                                   clv["frequency"],
                                   clv["recency"],
                                   clv["T"],
                                   clv["monetary_value"],
                                   time=3,
                                   freq='D',
                                   discount_rate=0.01)
clv.sort_values('3_months_clv',ascending=False).head()

,frequency,recency,T,monetary_value,expected_purc_3_months,3_months_clv
CustomerID,,,,,,
16446.0,1.0,205.0,205.0,168469.600000,0.662481,535689.563700
14646.0,44.0,353.0,354.0,6342.774091,9.893157,62648.940197
18102.0,25.0,367.0,367.0,9349.477200,5.537664,52424.706985
17450.0,26.0,359.0,367.0,7398.536538,5.750623,43026.334891
15749.0,1.0,97.0,332.0,21535.900000,0.382993,39590.283689
